In [1]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

In [5]:
# recursively loop all the files under templates folder
# and read their content
templates = {}
for root, dirs, files in os.walk("templates/billing"):
    for file in files:
        with open(os.path.join(root, file), "r") as f:
            templates[os.path.join(root, file)] = f.read()

In [7]:
# create the prompt with all the templates
boilerplate = ""
for index, template in enumerate(templates):
    boilerplate += f"Start of Boilerplate #{index+1}: {template}\n"
    boilerplate += f"{templates[template]}\n"
    boilerplate += f"End of Boilerplate #{index+1}: {template}\n\n"


In [8]:
print(boilerplate)

Start of Boilerplate #1: templates/billing/billing-data.tsx
import {
  Tooltip,
  TooltipContent,
  TooltipProvider,
  TooltipTrigger,
} from '@ui/core/shadcn/tooltip'

import { Label } from '@ui/core/shadcn/label'
import React from 'react'
import { TbClockDollar } from 'react-icons/tb'
import { TbFileDollar } from 'react-icons/tb'

export default function BillingData() {
  return (
    <div className="flex flex-col items-end justify-center gap-2">

    </div>
  )
}

End of Boilerplate #1: templates/billing/billing-data.tsx

Start of Boilerplate #2: templates/billing/billing.tsx
import BillingData from '@app/(protected)/case-profile/[id]/billing/billing-data'
import React from 'react'
import Tabs from '@common/components/tabs'
import { TbClockDollar } from 'react-icons/tb'
import { TbFileDollar } from 'react-icons/tb'

interface BillingProps {
  id: string
  children: React.ReactNode
}

export default function Billing({ id, children }: BillingProps) {
  const billingTabs = [
    {
    

In [9]:
from langchain.prompts import ChatPromptTemplate

template = """Update the code below with the following changes: 
 Add total time and total billing on the top right portion of the screen. 

-----------------
{boilerplate}
-----------------

Respond immediately in the following format:
Filename: [filename]
Code: [code] 
"""
prompt = ChatPromptTemplate.from_template(template)


In [46]:
from langchain.prompts import ChatPromptTemplate

template = """Generate {count} source code using next.js and prisma 
based on the given {count} boilerplate below. 
Follow the boilerplate markers to generate the multiple code. 

-----------------
{boilerplate}
-----------------

Replace the table name case to department.

Replace the fields of the boilerplate with to include the new fields below:
  id         
  name        
  type       
  category    
  description 
  isRequired 


Respond immediately in the following format:
Filename: [filename]
Code: [code] 
"""
prompt = ChatPromptTemplate.from_template(template)


In [47]:
print(len(templates))

3


In [10]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-4o")

In [11]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | openai_chat_model | StrOutputParser()

In [12]:
result = chain.invoke({"boilerplate": {boilerplate}, "count": len(templates)})

In [13]:
print(result)

Filename: templates/billing/billing-data.tsx
Code: 
```tsx
import {
  Tooltip,
  TooltipContent,
  TooltipProvider,
  TooltipTrigger,
} from '@ui/core/shadcn/tooltip'

import { Label } from '@ui/core/shadcn/label'
import React from 'react'
import { TbClockDollar } from 'react-icons/tb'
import { TbFileDollar } from 'react-icons/tb'

export default function BillingData() {
  return (
    <div className="flex flex-col items-end justify-center gap-2">
      <div className="flex flex-col items-end">
        <div className="flex items-center justify-between w-full">
          <span className="text-xl font-semibold">Total Time:</span>
          <span className="text-xl">00:00</span> {/* Placeholder for total time */}
        </div>
        <div className="flex items-center justify-between w-full">
          <span className="text-xl font-semibold">Total Billing:</span>
          <span className="text-xl">$0.00</span> {/* Placeholder for total billing */}
        </div>
      </div>
    </div>


In [14]:
files = result.split("Filename: ")
for file in files:
    if not file:
        continue
    filename, code = file.split("Code:")
    output_file = filename.strip().replace("templates/","output/")
    # add directory if not exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    with open(output_file, "w") as f:
        f.write(code)
        print(f"File {filename} has been generated.")

File templates/billing/billing-data.tsx
 has been generated.
File templates/billing/billing.tsx
 has been generated.


In [10]:
# recursively loop all the files under templates folder
# and read their content
file = "output/app/(protected)/departments/add-department.tsx"
with open(file, "r") as f:
    code = f.read()
code = "Filename:" + file +"\n\n" + code

In [ ]:
# create the prompt with all the templates
boilerplate = ""
for index, template in enumerate(templates):
    boilerplate += f"Start of Boilerplate #{index+1}: {template}\n"
    boilerplate += f"{templates[template]}\n"
    boilerplate += f"End of Boilerplate #{index+1}: {template}\n\n"

In [11]:
from langchain.prompts import ChatPromptTemplate

template = """Fix the warnings that are present in the given source code below.

-----------------
{code}
-----------------

web:lint: 5:20  Warning: 'Division' is defined but never used.  no-unused-vars
web:lint: 5:30  Warning: 'Nature' is defined but never used.  no-unused-vars
web:lint: 5:38  Warning: 'SystemCodes' is defined but never used.  no-unused-vars
web:lint: 5:51  Warning: 'User' is defined but never used.  no-unused-vars
web:lint: 25:10  Warning: 'DatePicker' is defined but never used.  no-unused-vars
web:lint: 48:3  Warning: 'natures' is defined but never used.  no-unused-vars
web:lint: 49:3  Warning: 'statuses' is defined but never used.  no-unused-vars
web:lint: 50:3  Warning: 'priorities' is defined but never used.  no-unused-vars
web:lint: 52:3  Warning: 'divisions' is defined but never used.  no-unused-vars
web:lint: 53:3  Warning: 'users' is defined but never used.  no-unused-vars


Respond immediately in the following format:
Filename: [filename]
Code: [code] 
"""
prompt = ChatPromptTemplate.from_template(template)

In [2]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo")

In [3]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | openai_chat_model | StrOutputParser()

In [6]:
result = chain.invoke({})

In [7]:
print(result)

Filename: AddDepartment.tsx
Code: 
```tsx
{' \n//start of the code below, include use client\n\nimport { Card, CardContent } from \'@ui/core/shadcn/card\'\nimport { Category, Division, Nature, SystemCodes, User } from \'@db/prisma\'\nimport {\n  Form,\n  FormControl,\n  FormField,\n  FormItem,\n  FormLabel,\n  FormMessage,\n} from \'@ui/core/shadcn/form\'\nimport {\n  Sheet,\n  SheetContent,\n  SheetHeader,\n  SheetTitle,\n  SheetTrigger,\n} from \'@ui/core/shadcn/sheet\'\n\nimport { AiOutlineFileAdd } from \'react-icons/ai\'\nimport { Button } from \'@ui/core/shadcn/button\'\nimport { Combobox } from \'@ui/core/shadcn/combobox\'\nimport { DatePicker } from \'@ui/core/shadcn/datepicker\'\nimport { Input } from \'@ui/core/shadcn/input\'\nimport React from \'react\'\nimport { getErrorMessage } from \'@common/lib/handle-error\'\nimport { toast } from \'sonner\'\nimport { useForm } from \'react-hook-form\'\nimport { useRouter } from \'next/navigation\'\nimport { z } from \'zod\'\nimport { 

In [15]:
files = result.split("Filename: ")
for file in files:
    if not file:
        continue
    filename, code = file.split("Code:")
    output_file = filename.strip().replace("templates/","output/")
    # add directory if not exists
    #os.makedirs(os.path.dirname(output_file), exist_ok=True)
    with open(output_file, "w") as f:
        f.write(code)
        print(f"File {filename} has been generated.")

File output/app/(protected)/departments/add-department.tsx
 has been generated.


In [12]:
# recursively loop all the files under templates folder
# and read their content
import re
file = "debug/lint.txt"
with open(file, "r") as f:
    lints = f.read()
# split into "web:lint"
lints = lints.split("\n")
warnings = {}
# check if there are any warnings for the files generated
if len(lints) > 1:
    for lint in lints:        
        # if this refers to new file, get the filename
        lint = lint.strip()
        # lint = re.sub(r'\x1b\[.*?m', '', lint)        
        if lint.startswith("./"):
            filename = lint
        # if blank, disregard
        elif not lint.strip():
            filename = ""
            continue
        elif filename:
            # add the rows to the warnings
            # if dict does not exist, add it else, append to array of warnings
            if filename not in warnings:
                warnings[filename] = [lint]
            else:
                warnings[filename] += [lint]
    print(warnings)


        
        


{'./src/app/(protected)/admin/users/table.tsx': ["39:5  Warning: 'isError' is assigned a value but never used.  no-unused-vars", "41:5  Warning: 'error' is assigned a value but never used.  no-unused-vars", "180:18  Warning: 'row' is defined but never used.  no-unused-vars", "196:18  Warning: 'row' is defined but never used.  no-unused-vars", "198:33  Warning: 'startEditTransition' is assigned a value but never used.  no-unused-vars", "199:35  Warning: 'startDeleteTransition' is assigned a value but never used.  no-unused-vars"], './src/app/(protected)/ai/ensemble/compose/page.tsx': ["3:8  Warning: 'MiniMapNode' is defined but never used.  no-unused-vars", "4:8  Warning: 'ShapeNode' is defined but never used.  no-unused-vars", "5:3  Warning: 'ShapeNodeData' is defined but never used.  no-unused-vars", "7:8  Warning: 'Sidebar' is defined but never used.  no-unused-vars", "10:17  Warning: 'useCallback' is defined but never used.  no-unused-vars", "14:3  Warning: 'MiniMap' is defined but 

In [6]:
warnings

{}

In [30]:
from langchain.prompts import ChatPromptTemplate

template = """You are a systems analyst. Your task is to ask for the following information:
1. model: [model or entity name]
2. fields: [fields of the model]
3. output: [folder location of the project]

below are the information you know

model: 'department'
fields: 'name, categorty, description, isRequired'
output: '/src/out'

If all information is complete, respond with "Requirements complete."
Ask one question with missing information in order.
"""
prompt = ChatPromptTemplate.from_template(template)

In [32]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | openai_chat_model | StrOutputParser()

In [33]:
result = chain.invoke({})
print(result)

What is the folder location for the project output?


In [39]:
import os
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

def generate_code(requirements):
    print("Generating code...")
    templates = {}
    for root, dirs, files in os.walk("templates/app/(protected)/cases"):
        for file in files:
            with open(os.path.join(root, file), "r") as f:
                templates[os.path.join(root, file)] = f.read()

    # create the prompt with all the templates
    boilerplate = ""
    for index, template in enumerate(templates):
        boilerplate += f"Start of Boilerplate #{index+1}: {template}\n"
        boilerplate += f"{templates[template]}\n"
        boilerplate += f"End of Boilerplate #{index+1}: {template}\n\n"

        
    template = """Generate {count} source code using next.js and prisma 
    based on the given {count} boilerplate below. 
    Follow the boilerplate markers to generate the multiple code. 

    -----------------
    {boilerplate}
    -----------------

    Replace the table name case to {model}.

    Replace the fields of the boilerplate with to include the new fields below:
      {fields_newline} 


    Respond immediately in the following format:
    Filename: [filename]
    Code: [code] 
    """
    prompt = ChatPromptTemplate.from_template(template)

    openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo")
    chain = prompt | openai_chat_model | StrOutputParser()
    print(f"Invoking model with {len(templates)} templates...")
    result = chain.invoke({"boilerplate": {boilerplate}, 
                           "count": len(templates),
                           "model": requirements["model"],
                           "fields_newline": "\n".join(requirements["fields"])})
    print("Saving to file...")
    # save boilerplate to file
    with open("debug/boilerplate.txt", "w") as f:
        f.write(boilerplate)
    files = result.split("Filename: ")
    for file in files:
        if not file:
            continue
        filename, code = file.split("Code:")
        output_file = filename.strip().replace("templates/","output/")
        # add directory if not exists
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        with open(output_file, "w") as f:
            f.write(code)
            print(f"File {filename} has been generated.")

In [41]:
requirements = {
    "model": "User",
    "fields": ["id", "name", "email", "password", "created_at", "updated_at"]
}
generate_code(requirements)

Generating code...
Invoking model with 3 templates...
Saving to file...
File templates/app/(protected)/users/user-page.tsx
 has been generated.
File templates/app/(protected)/users/user-search-table.tsx
 has been generated.
File templates/app/(protected)/users/add-user.tsx
 has been generated.
